<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/Generate_Chicago_Traces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary packages
import os, os.path
from glob import glob
from shutil import copyfile

import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
files = glob("drive/My Drive/SatTemp/chicago_synthpop_V2/*.csv.gz")
print(files)

['drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz']


In [6]:
!rm -rf synthpop
if not os.path.exists('synthpop'):
  os.mkdir('synthpop')

In [7]:
# copy these files from Google Drive to colab disk
for f in files:
  print(f)
  copyfile(f, 'synthpop/' + os.path.basename(f))

drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz


In [10]:
# look at people
persons_df = pd.read_csv('synthpop/persons_v2.csv.gz')
persons_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927761 entries, 0 to 2927760
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   numeric_id                int64  
 1   person_id                 object 
 2   hh_id                     float64
 3   relate                    float64
 4   sex                       int64  
 5   age                       int64  
 6   gq_id                     float64
 7   school_id                 float64
 8   work_id                   float64
 9   daycare_id                float64
 10  gym_id                    float64
 11  hospital_id               float64
 12  jail                      float64
 13  jail_id                   int64  
 14  rank                      int64  
 15  weekday_schedule_id_list  object 
 16  weekend_schedule_id_list  object 
 17  otherhh_id                object 
 18  home_restaurant_id        object 
 19  recreation_id             object 
 20  grocery_id              

In [12]:
# places
places_df = pd.read_csv('synthpop/places_v2.csv.gz')
places_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210722 entries, 0 to 1210721
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   numeric_id  1210722 non-null  float64
 1   place_id    1210722 non-null  object 
 2   place_type  1210722 non-null  object 
 3   name        1093276 non-null  object 
 4   persons     1077336 non-null  float64
 5   latitude    1210668 non-null  float64
 6   longitude   1210668 non-null  float64
 7   zipcode     1204493 non-null  float64
 8   zip_grp     1194456 non-null  float64
 9   rank        1210722 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 92.4+ MB


In [15]:
# activities
activity_df = pd.read_csv('synthpop/activities_v2.csv.gz')
activity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239781 entries, 0 to 239780
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Act_SlNo        239781 non-null  int64 
 1   start_time      239781 non-null  int64 
 2   stop_time       239781 non-null  int64 
 3   Social_Act_Loc  239781 non-null  object
 4   schedule_id     239781 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.1+ MB


In [13]:
places_df.head()

,numeric_id,place_id,place_type,name,persons,latitude,longitude,zipcode,zip_grp,rank
0,1.0,10049854,Hospital,NaN,NaN,41.900780,-87.699604,60622.0,2.0,94
1,2.0,10049882,Hospital,NaN,NaN,41.558686,-87.798062,60477.0,0.0,80
2,3.0,10049883,Hospital,NaN,NaN,41.895282,-87.619157,60611.0,2.0,81
3,4.0,10051140,Hospital,NaN,NaN,41.861271,-87.834218,60141.0,0.0,81
4,5.0,10052705,Hospital,NaN,NaN,41.895516,-87.619425,60611.0,2.0,82
